In [15]:
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch
import bisect


from string import ascii_lowercase
from scipy.linalg import block_diag
from itertools import chain, combinations
from collections import OrderedDict
from math import pi
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.io import output_notebook
from utilities.colors import get_colors

In [16]:
# This is Tim's code

def simulate_normal_data(nobs, num_members, correlations, seed):
    """Simulate normal data with specific group correlation structure. 
    
    Args:
        nobs (int): Number of observations.
        num_members (list): List of integers, specifying the numbers of correlated
            covariates in a group. Number of groups = len(num_members). Length of
            num_members has to be equal to length of correlations.
        correlations (list): List of floats in [0, 1].
        seed (int): Random number seed.

    Returns:
    
    
    """
    np.random.seed(seed)
    dimension = int(sum(num_members))
    cov = create_block_correlation_matrix(num_members, correlations)
    X = np.random.multivariate_normal(mean=np.zeros(dimension), cov=cov, size=nobs)
    return X

def create_block_correlation_matrix(num_members, correlations):
    """Create correlation matrix with block structure.
    
    Args:
        num_members (list): List of integers, specifying the numbers of correlated
            covariates in a group. Number of groups = len(num_members). Length of
            num_members has to be equal to length of correlations.
        correlations (list): List of floats in [0, 1].
    
    """
    matrices = [create_correlation_matrix(dim, corr) for dim, corr in zip(num_members, correlations)]
    block_matrix = block_diag(*matrices)
    return block_matrix

def create_correlation_matrix(dim, correlation):
    """Create simple correlation matrix.
    
    Create simple correlation matrix with size (dim, dim)
    and off-diagonal correlation given by correlation.
    
    Args:
        dim (int): Dimension.
        correlation (float): Positive float representing the off-diagonal
            correlation.
    
    """
    mat = np.full((dim, dim), correlation)
    np.fill_diagonal(mat, 1)
    return mat

def plot_correlation_heatmap(corrmat, figsize=(11, 9)):
    """Plot correlation matrix as heatmap."""
    sns.set(style="white")
    mask = np.triu(np.ones_like(corrmat, dtype=np.bool))
    f, ax = plt.subplots(figsize=figsize)
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corrmat, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

def find_clusters_correlation_matrix(
        corrmat,
        remove_uncorrelated_features=False,
        threshold=0.05,
        return_clustered_corrmat=False
):
    """Find clusters in correlation matrix using hierachical clustering.
    
    Args:
        corrmat (np.array): The correlation matrix.
        remove_uncorrelated_features: Should features with maximum correlation
            less than ``threshold`` be dropped.
        threshold (float): Positive threshold in (0, 1) determining the number of
            features to drop if ``remove_uncorrelated_features`` is True.
        return_clustered_corrmat (bool): Should the reordered corrmat be returned.
        
    Returns:
        out (dict): Dictionary containing:
            - index (np.array): 
                Index to reorder features
            - relevant_features (np.array):
                Index to reorder features where irrelevant features are dropped, if
                return_uncorrelated_features is True.
            - cm (np.array):
                Clustered correlation matrix, if return_clustered_corrmat is True.
            - cm_relevant (np.array):
                Clustered correlation matrix where irrelevant features are dropped, if
                return_uncorrelated_features and return_clustered_corrmat is True.
    """
    # hierachical clustering
    distance = sch.distance.pdist(corrmat)
    linkage = sch.linkage(distance, method='complete')
    index = sch.fcluster(linkage, 0.5 * distance.max(), 'distance')
    index = np.argsort(index)
    
    # reorder features
    cm = corrmat[:, index][index, :].copy()
    
    out = {}
    out['index'] = index
    
    if remove_uncorrelated_features:
        relevant_features = compute_relevant_features(cm, threshold)
        out['relevant_features'] = relevant_features
        
    if return_clustered_corrmat:
        out['cm'] = cm
        if remove_uncorrelated_features:
            cm_relevant = cm[:, relevant_features][relevant_features, :].copy()
            out['cm_relevant'] = cm_relevant

    return out


def compute_relevant_features(corrmat, threshold):
    """Compute indices of relevant features.
    
    Computes which features have a correlation with any other feature
    greater than ``threshold``.
    
    Args:
        corrmat (np.array): The correlation matrix.
        threshold (float): The threshold. Must be in (0, 1).
    
    Returns:
        relevant_features (np.array): Integer array containing the relevant features.
        
    """
    cm_without_diagonal = skip_diag(corrmat)
    max_correlation = cm_without_diagonal.max(axis=0)
    relevant_features = np.where(max_correlation > threshold)[0]
    return relevant_features


def skip_diag(A):
    """Skip diagonal elements of matrix.
    
    Args:
        A (np.array): 2 dim. array of which we want to drop the diagonal elements.
    
    Returns:
        out (np.array): The new 2 dim. array without diagonal elements.
    
    Example:
    > A = np.arange(9).reshape((3, 3))
    > A
    array([[0, 1, 2],
          [3, 4, 5],
          [6, 7, 8]])
    > skip_diag(A)
    array([[3, 1, 2],
           [6, 7, 5]])
           
    """
    AA = A.copy().T
    out = AA[~np.eye(A.shape[0],dtype=bool)].reshape(A.shape[0],-1)
    return out.T

In [42]:
#Code that produces heatmaps

def make_optimal_heatmap(varnames, df, order_dict, plot_width=600, plot_heigth=600):
    """ Create optimal heatmap with variables in varnames.
    Args:
        df (pd.DataFrame): raw data to base the plot on.
        varnames (list): potentially unsorted list of variables.
        plot_width (int): width of plot in pixels.
        plot_heigth (int): height of plot in pixels.
    
    Returns:
        hm: heatmap with optimal order
        
        
    """
    
    varlist = varnames.copy()
    varlist.sort()
    
    order = order_dict[tuple(varlist)]
    data = df[order]
    
    hm = make_basic_heatmap(data)
    
    return hm

def make_basic_heatmap(data, plot_width=600, plot_heigth=600):
    
    """Create a heatmap from a DataFrame using the bokeh library. Most of this code is taken 
    from the answer by stackoverflow user "Philly" to
    https://stackoverflow.com/questions/39191653/python-bokeh-how-to-make-a-correlation-plot
    
    Args:
        df (pd.DataFrame): raw data to base the plot on.
        plot_width (int): width of plot in pixels.
        plot_heigth (int): height of plot in pixels.
        
    Returns:
        heatmap.
        
    """
    
    
    # we want an odd number to ensure 0 correlation is a distinct color
    #colors = list(reversed(RdBu[9]))  
    r = get_colors("red", 8)
    b = get_colors("blue", 9)
    
    colors = b + r[::-1]

    labels = data.columns
    nlabels = len(labels)
    
    corr_mat = data.corr()

    hm = figure(plot_width=600, plot_height=600,
               x_range=(0,nlabels), y_range=(nlabels,0),
               title="Heatmap of Correlations",
               toolbar_location=None, tools='')

    hm.xgrid.grid_line_color = None
    hm.ygrid.grid_line_color = None
    hm.xaxis.major_label_orientation = pi/4
    hm.yaxis.major_label_orientation = pi/4

    top, bottom, left, right = _set_bounds(nlabels)  # creates sqaures for plot
    color_list = _set_colors(corr_mat.values.flatten(), colors)

    hm.quad(top=top, bottom=bottom, left=left,
           right=right, line_color='white',
           color=color_list)

    # Set ticks with labels
    ticks = [tick+0.5 for tick in list(range(nlabels))]
    tick_dict = OrderedDict([[tick, labels[ii]] for ii, tick in enumerate(ticks)])
    # Create the correct number of ticks for each axis 
    hm.xaxis.ticker = ticks
    hm.yaxis.ticker = ticks
    # Override the labels 
    hm.xaxis.major_label_overrides = tick_dict
    hm.yaxis.major_label_overrides = tick_dict

    # Setup color bar
    mapper = LinearColorMapper(palette=colors, low=-1, high=1)
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
    hm.add_layout(color_bar, 'right')
    
    return hm

def _set_bounds(nlabels):
        """Gets bounds for quads with n features"""
        bottom = list(chain.from_iterable([[ii]*nlabels for ii in range(nlabels)]))
        top = list(chain.from_iterable([[ii+1]*nlabels for ii in range(nlabels)]))
        left = list(chain.from_iterable([list(range(nlabels)) for ii in range(nlabels)]))
        right = list(chain.from_iterable([list(range(1,nlabels+1)) for ii in range(nlabels)]))
        return top, bottom, left, right

def _set_colors(corr_array, colors):
    """Aligns color values from palette with the correlation coefficient values"""
    ccorr = np.arange(-1, 1, 1/(len(colors)/2))
    color = []
    for value in corr_array:
        ind = bisect.bisect_left(ccorr, value)
        color.append(colors[ind-1])
    return color

In [43]:
# Code to create dictionary

def create_order_dict(df):
    """Return a dictionary with alphabetically sorted  tuples of subsets of variable names as keys
    and the optimal ordering of variables for a heatmap as tuple values.
    
    Args:
        df (pd.DataFrame): original dataset.
    
    Returns:
        order_dict (dict): dictionary with subsets as keys and optimal orders as values.
    
    """
    
    
    columns = list(df.columns)
    
    keys = _find_subsets(columns)  
    
    values = [_find_heatmap_order(list(var), df) for var in keys]
    
    order_dict = dict(zip(keys, values))
    
    return order_dict
    
    



def _find_heatmap_order(varlist, df):
    """Calculate the optimal order for a subset of variables for the sake of 
    making a heatmap with these variables given an original dataframe.
    
    Args:
        varlist (list): list of variables to consider.
        df (pd.DataFrame): original dataset.
        
    Returns:
        name_order (list): optimal order for making a heatmap from varlist.
    
    """
        
    data = df[varlist]
    corr_mat = np.array(data.corr())
    order = find_clusters_correlation_matrix(corr_mat)["index"]
    
    name_order = list(pd.Series(varlist)[order])
    
    return name_order
    


def _find_subsets(masterset):
    """Find list of all (alphabetically sorted) unordered subsets.
    
    Args:
        masterset (list): set to take subsets from.
    
    Returns:
        subsets (list): list of tuples, where each tuple is a subset.
    
    """
    
    masterlist = masterset.copy()
    masterlist.sort()
    subsets=[]
    
    for m in range(2,len(masterlist)+1):
        subsets = subsets + [tuple(item) for item in _find_subsets_of_size(masterlist,m)]
    
    return subsets
    

def _find_subsets_of_size(masterset, size):
    """ Compute all subsets of a set of a given size.
    """
    comb = combinations(masterset, size)
    return [list(item) for item in comb]
    
    return 


# Example

In [56]:
#Create example data set
col_names = list(ascii_lowercase)[:15]

orig_data = simulate_normal_data(nobs=100,
                            num_members=[3,4,3,5], 
                            correlations=[0.2, 0.35, 0.71, 0.9],
                            seed=500)

group1 = list(ascii_lowercase)[:3]
group2 = list(ascii_lowercase)[3:10]
group3 = list(ascii_lowercase)[10:15]


num_features = orig_data.shape[1]
shuffle_index = np.random.permutation(np.arange(num_features))
shuffle_data = orig_data[:, shuffle_index]

df = pd.DataFrame(shuffle_data, columns=col_names)

In [51]:
# Create dictionary
order_dict = create_order_dict(df)

In [52]:
hm = make_optimal_heatmap(["e", "g", "h", "a", "b"], df, order_dict)

output_notebook()
show(hm)

Loading BokehJS ...

In [57]:
group1

['a', 'b', 'c']

In [58]:
group2

['d', 'e', 'f', 'g', 'h', 'i', 'j']

In [59]:
group3

['k', 'l', 'm', 'n', 'o']